In [1]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from scripts.point_forecast_prediction import prepare_training_data, train_regression_tree, forecast_point_generation

In [2]:
# Generate simulated data
seed = 42
hours = 24
np.random.seed(seed)
timestamps = pd.date_range("2022-06-01", periods=hours, freq="h", tz="UTC")
df = pd.DataFrame({"timestamp": timestamps})
df["temperature"] = 25 + 5 * np.sin(np.pi * df.index / hours)
df["cloudiness"] = np.clip(np.random.normal(0.5, 0.3, size=hours), 0, 1)
df["humidity"] = 60 + 10 * np.random.rand(hours)
df["Ppccs"] = 5000 * np.clip(np.sin(np.pi * df.index / hours), 0, 1)
df["Pm"] = df["Ppccs"] * (1 - df["cloudiness"] * 0.6)

In [3]:
df.head()

,timestamp,temperature,cloudiness,humidity,Ppccs,Pm
0,2022-06-01 00:00:00+00:00,25.000000,0.649014,65.924146,0.000000,0.000000
1,2022-06-01 01:00:00+00:00,25.652631,0.458521,60.464504,652.630961,473.084074
2,2022-06-01 02:00:00+00:00,26.294095,0.694307,66.075449,1294.095226,754.995942
3,2022-06-01 03:00:00+00:00,26.913417,0.956909,61.705241,1913.417162,814.837550
4,2022-06-01 04:00:00+00:00,27.500000,0.429754,60.650516,2500.000000,1855.369019


In [4]:
# Prepare training data
features = ["temperature", "cloudiness", "humidity"]
X_train, y_train = prepare_training_data(df, features)

In [5]:
X_train.head()

,temperature,cloudiness,humidity
1,25.652631,0.458521,60.464504
2,26.294095,0.694307,66.075449
3,26.913417,0.956909,61.705241
4,27.500000,0.429754,60.650516
5,28.043807,0.429759,69.488855


In [6]:
X_train.shape

(23, 3)

In [7]:
y_train.head()

1    0.275112
2    0.416584
3    0.574145
4    0.257852
5    0.257855
Name: Lambda, dtype: float64

In [8]:
y_train.shape

(23,)

In [9]:
# Train regression tree
model = train_regression_tree(X_train, y_train)

In [10]:
# Generate point forecast for the next 24 hours
df_forecast = forecast_point_generation(model, df, features)

In [11]:
df_forecast.head()

,timestamp,temperature,cloudiness,humidity,Ppccs,Pm,Ppf
0,2022-06-01 00:00:00+00:00,25.000000,0.649014,65.924146,0.000000,0.000000,0.000000
1,2022-06-01 01:00:00+00:00,25.652631,0.458521,60.464504,652.630961,473.084074,476.101955
2,2022-06-01 02:00:00+00:00,26.294095,0.694307,66.075449,1294.095226,754.995942,702.473197
3,2022-06-01 03:00:00+00:00,26.913417,0.956909,61.705241,1913.417162,814.837550,820.807689
4,2022-06-01 04:00:00+00:00,27.500000,0.429754,60.650516,2500.000000,1855.369019,1851.053272
